In [ ]:
#######################################################################################################################
# Project: Deep Virtual Rapport Agent (data preprocessing)
#
#     Jan Ondras (jo951030@gmail.com)
#     Institute for Creative Technologies, University of Southern California
#     April-October 2019
#
#######################################################################################################################
# Annotate OpenFace features from the nvb dataset with ground-true head gestures.
#
#     Each frame will be annotated with ground-truth annotations of nod, shake, and tilt head gestures.
# 
#     Note: Make sure the unify_filenames.ipynb script was run!
#
#     Input features: dvra_datasets/nvb/openface_features 
#     Input annotations: dvra_datasets/nvb/original_data/head_gesture_annotations 
#     Output features: dvra_datasets/nvb/annotated_features
#######################################################################################################################

In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

# For each recording
#     Resample feature dataframe
#     Add first derivatives of selected features
#     Annotate frames
#     Save as new annotated dataframe, 0 => not a nod, 1 => nod

import os
import glob
import pandas as pd
import scipy.signal
from scipy import interpolate
import time
import pympi    # Import pympi to work with elan files

# Unified frame rate
FRAME_RATE = 30.

# Features whose first and second derivatives will be calculated
diff_selected_features = [
    ' pose_Tx', 
    ' pose_Ty', 
    ' pose_Tz', 

    ' pose_Rx', 
    ' pose_Ry', 
    ' pose_Rz',

    ' p_rx', 
    ' p_ry',
    ' p_rz'
    
    # add landmarks?
]

# For elan annotations
tier_name_to_head_gesture = {
    'Head Nod': 'nod', 
    'Head Shake': 'shake', 
    'Head Tilt': 'tilt'
}

input_features_dir = '/home/ICT2000/jondras/dvra_datasets/nvb/openface_features'
input_annotations_dir = '/home/ICT2000/jondras/dvra_datasets/nvb/original_data/head_gesture_annotations '
output_dir = '/home/ICT2000/jondras/dvra_datasets/nvb/annotated_features'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
start_time = time.time()    
cnt = 0
for feature_file in sorted(glob.glob(input_features_dir + '/*.csv')):
    
    sid = feature_file.split('/')[-1].split('.')[0]
    print(f'Processing SESID {sid}')
    
    org_df = pd.read_csv(feature_file)
    print(len(org_df))
#     print(len(org_df), org_df)
    
    # Resample feature dataframe to common frame rate, if needed
    csv_frame_rate = (org_df.shape[0] - 1) / np.sum(np.diff(org_df[' timestamp']))
    print(f'\tcsv frame rate: {csv_frame_rate}')
    if round(csv_frame_rate) == FRAME_RATE:
        print(f'\tNOT resampling')
        new_df = org_df.copy()
    else:
        print(f'\tREsampling from {csv_frame_rate} to {FRAME_RATE}')
        new_df = []
        timestamps_resampled = np.arange(0., org_df.iloc[-1][' timestamp'], step=1. / FRAME_RATE)
        for col_name in org_df.columns:            
            # Get interpolation function
            f = interpolate.interp1d(x=org_df[' timestamp'], y=org_df[col_name], kind='linear')
            new_df.append( f(timestamps_resampled) )
        # Does not work properly
#         n_resampled_points = int(1 + (len(org_df[' timestamp']) - 1) * FRAME_RATE / csv_frame_rate)
#         for col_name in org_df.columns:            
#             new_col = scipy.signal.resample(np.array(org_df[col_name]), num=n_resampled_points)
#             new_df.append(new_col)
        new_df = pd.DataFrame(np.array(new_df).T, columns=org_df.columns) 
                
    # Add first and second derivatives of selected features
    diff_features = dict()
    for feature_name in diff_selected_features:
        diff_features['diff_' + feature_name] =  np.diff(new_df[feature_name], prepend=new_df[feature_name][0])
        diff_features['diff2_' + feature_name] = np.diff(diff_features['diff_' + feature_name], 
                                                         prepend=diff_features['diff_' + feature_name][0])    
    new_df = new_df.assign(**diff_features)
    
    # Annotate frames
    
    # Add annotation columns for nod, shake, and tilt
    new_df = new_df.assign(nod=np.zeros(len(new_df), dtype=int))
    new_df = new_df.assign(shake=np.zeros(len(new_df), dtype=int))
    new_df = new_df.assign(tilt=np.zeros(len(new_df), dtype=int))
    
    eaf_obj = pympi.Elan.Eaf(f'{input_annotations_dir}/{sid}.eaf')
    for tier_name in eaf_obj.get_tier_names():
        # Ignore annotations other than head nods, shakes, and tilts
        if tier_name in tier_name_to_head_gesture.keys():
            head_gesture = tier_name_to_head_gesture[tier_name]
            anns = eaf_obj.get_annotation_data_for_tier(tier_name)
            # Iterate over annotation intervals (in milliseconds)
            for ann in anns:
                interval_begin = ann[0] / 1000.
                interval_end   = ann[1] / 1000.
                new_df[head_gesture] = pd.np.where(
                    (new_df[' timestamp'] >= interval_begin) & (new_df[' timestamp'] <= interval_end), 1, 
                    new_df[head_gesture])
#                 print(interval_begin, interval_end, head_gesture)
#                 print(np.where(np.array(new_df.nod) == 1))
#                 print(np.where(np.array(new_df.shake) == 1))
#                 print(np.where(np.array(new_df.tilt) == 1))
    
    # Save as new annotated dataframe
#     print(new_df)
    new_df.to_csv(f'{output_dir}/{sid}.csv', index=False)
    cnt += 1
    print(f'Time taken: {time.time() - start_time} s\n')        
#     break

print(f'\nGenerated {cnt} annotated feature files.')

Processing SESID RAW01F1
4036
	csv frame rate: 29.969918668993948
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 4.974734544754028 s

Processing SESID RAW02M0
20532
	csv frame rate: 29.970031428317014
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 28.503784894943237 s

Processing SESID RAW03F0
5145
	csv frame rate: 29.97005325161095
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 34.46823334693909 s

Processing SESID RAW04M1
4773
	csv frame rate: 29.96997977717207
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 39.900651931762695 s

Processing SESID RAW05M0
6382
	csv frame rate: 29.969987741471872
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 47.22550392150879 s

Processing SESID RAW06F1
5678
	csv frame rate: 29.969961409121385
	NO